Association Rule Mining is a technique to find sets of items that are frequently purchased together. I have tried to apply this technique on my personal groceries data that I have gathered for over 2+ years through digital bills of my purchases.

Apriori Algorithm is a ML algorithm that helps us to find items that are frequently purchased together through association rules.

In [ ]:
!pip install mlxtend

In [ ]:
#Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#Load the products dataset
products_df = pd.read_excel('/content/my_groceries_items.xlsx')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Data Exploration**

In [ ]:
#Display Products data
products_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Produkt,Stückzahl,Preis,Mehrwertsteuer,Datum,Zeit,Transaktionsnummer,Filale
0,Vegane KuchenVanille,1.000,1.89,A,14.04.21,19:15:00,38013,Magdeburg
1,5 Croissants gefüllt,1.000,0.99,A,14.04.21,19:15:00,38013,Magdeburg
2,Orangenfruchtsaft,1.000,1.39,B,14.04.21,19:15:00,38013,Magdeburg
3,Rahmspinat,1.000,0.59,A,14.04.21,19:15:00,38013,Magdeburg
4,Bananen kg,0.866,0.86,A,14.04.21,19:15:00,38013,Magdeburg
...,...,...,...,...,...,...,...,...
1783,Kidney-Bohnen,2.000,1.38,A,03.02.24,15:30:00,728142,Aschaffenburg
1784,Erdnusskerne,2.000,2.58,A,03.02.24,15:30:00,728142,Aschaffenburg
1785,Fl. Handseife Senset,1.000,0.65,B,03.02.24,15:30:00,728142,Aschaffenburg
1786,NatureDuschgelCitrus,1.000,1.25,B,03.02.24,15:30:00,728142,Aschaffenburg


**Description of Products Dataset:**

Produkt: Name of the Product in German. Some correspond to the Rabatt (discount) obtained due to app coupon and Pfandrückgabe (deposit returned on bottles)

Stückzahl: Quantity of the product purchased. Some values will be decimals which implies quantity in kilograms for items such as Fruits, Vegetables

Preis: Price of the product

Mehrwertsteuer: Category of tax levied on the product. Some values are Keiner means no tax, which is related to the Rabatt and Pfandrückgabe.

Datum: Date on which product is purchased

Zeit: Time at which product is purchased

Transaktionsnummer: Transaction number for the product

Filale: City in which store is located

In [ ]:
#Check datatypes of Products dataframe
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1788 entries, 0 to 1787
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Produkt             1788 non-null   object 
 1   Stückzahl           1788 non-null   float64
 2   Preis               1788 non-null   float64
 3   Mehrwertsteuer      1788 non-null   object 
 4   Datum               1788 non-null   object 
 5   Zeit                1788 non-null   object 
 6   Transaktionsnummer  1788 non-null   int64  
 7   Filale              1788 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 111.9+ KB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Data Cleaning**

In [ ]:
#Removing App Rabatt and Pfandrückgabe as they are not products
products_df = products_df[products_df.Produkt != 'App Rabatt']
products_df = products_df[products_df.Produkt != 'Pfandrückgabe']
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1501 entries, 0 to 1787
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Produkt             1501 non-null   object 
 1   Stückzahl           1501 non-null   float64
 2   Preis               1501 non-null   float64
 3   Mehrwertsteuer      1501 non-null   object 
 4   Datum               1501 non-null   object 
 5   Zeit                1501 non-null   object 
 6   Transaktionsnummer  1501 non-null   int64  
 7   Filale              1501 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 105.5+ KB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#Getting Transactions done in Magdeburg
basket_Magdeburg = (products_df[products_df['Filale'] =="Magdeburg"]
		.groupby(['Transaktionsnummer', 'Produkt'])['Stückzahl']
		.sum().unstack().reset_index().fillna(0)
		.set_index('Transaktionsnummer'))

#Getting Transactions done in Aschaffenburg
basket_Aschaffenburg = (products_df[products_df['Filale'] =="Aschaffenburg"]
		.groupby(['Transaktionsnummer', 'Produkt'])['Stückzahl']
		.sum().unstack().reset_index().fillna(0)
		.set_index('Transaktionsnummer'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Defining the one hot encoding function
def hot_encode(x):
	if(x<= 0):
		return 0
	if(x>= 1):
		return 1

# Encoding the Magdeburg and Aschaffenburg datasets
basket_encoded = basket_Magdeburg.applymap(hot_encode)
basket_Magdeburg = basket_encoded

basket_encoded = basket_Aschaffenburg.applymap(hot_encode)
basket_Aschaffenburg = basket_encoded

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
basket_Magdeburg

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Produkt,Zahncr. Kräut. 125ml,reis-fit Feel. Bohnen,5 Croissants gefüllt,ACE Vitamin Getränk,Abfallsäcke 601 25er,Active Sandwi. Toast,Active Sandwi. Toast,Al Basha Baklava,Alkoholfr. Bier,All-in-one Schutz,...,funny frisch Orient,funny frisch Orient.,funny-fr.Kicher.Papr,funny-frisch Orienta,funny-frisch Paprika,funnyfr. ungar.,funnyfrisch oriental,meridol Zahnbürste,reis-fit Feel.Linsen,reis-fit Kichererbse
Transaktionsnummer,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3360,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15296,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602644,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
616590,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
620119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Filling any NAN values in Magdeburg dataset with 0
basket_Magdeburg = basket_Magdeburg.fillna(0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
basket_Magdeburg

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Produkt,Zahncr. Kräut. 125ml,reis-fit Feel. Bohnen,5 Croissants gefüllt,ACE Vitamin Getränk,Abfallsäcke 601 25er,Active Sandwi. Toast,Active Sandwi. Toast,Al Basha Baklava,Alkoholfr. Bier,All-in-one Schutz,...,funny frisch Orient,funny frisch Orient.,funny-fr.Kicher.Papr,funny-frisch Orienta,funny-frisch Paprika,funnyfr. ungar.,funnyfrisch oriental,meridol Zahnbürste,reis-fit Feel.Linsen,reis-fit Kichererbse
Transaktionsnummer,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3360,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15296,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602644,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
616590,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
620119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
basket_Aschaffenburg

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Produkt,Abfallsäcke 60l 25er,Active Sandwi. Toast,Aioli,Anti Hornhaut Socken,Antip.gr.Pfefferonen,Asiat.Fertigg.Curry,Auberginen kg,Avocado,Avocado Stk.,Ayran Joghurtgetränk,...,Zitrone Bio Stk,Zitronensaft,Zucchini kg,Zuckermais gekocht,Zuckermelo. Stk,Zwiebeln gelb,Zwiebeln rot,funny-fr.Kicher.Papr,funnyfr. ungar.,iglo Rahmspinat
Transaktionsnummer,,,,,,,,,,,,,,,,,,,,,
399831,1,0,0,0,0,0,0.0,0,0,0,...,0,0,0.0,0,0,0,0,1,0,0
472857,0,0,0,0,0,0,0.0,0,0,0,...,0,0,0.0,1,0,0,0,0,0,0
486341,0,0,0,1,1,0,0.0,0,0,0,...,0,0,0.0,1,0,0,0,0,0,0
494783,0,0,0,0,0,0,0.0,0,0,0,...,0,0,0.0,0,1,0,0,0,0,0
508595,0,0,0,0,0,0,0.0,0,0,0,...,0,0,0.0,0,0,1,0,0,0,1
551374,0,0,0,0,0,0,0.0,0,0,0,...,0,0,0.0,0,0,0,0,0,0,0
576802,0,0,0,0,0,0,0.0,0,0,0,...,0,0,0.0,0,0,0,0,0,0,0
582831,0,1,0,0,0,0,0.0,0,0,0,...,0,0,0.0,0,0,1,0,0,0,0
597009,0,0,0,0,0,0,0.0,0,0,0,...,1,0,NaN,0,0,0,0,0,0,0


In [ ]:
#Filling any NAN values in Aschaffenburg dataset with 0
basket_Aschaffenburg = basket_Aschaffenburg.fillna(0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
basket_Aschaffenburg

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Produkt,Abfallsäcke 60l 25er,Active Sandwi. Toast,Aioli,Anti Hornhaut Socken,Antip.gr.Pfefferonen,Asiat.Fertigg.Curry,Auberginen kg,Avocado,Avocado Stk.,Ayran Joghurtgetränk,...,Zitrone Bio Stk,Zitronensaft,Zucchini kg,Zuckermais gekocht,Zuckermelo. Stk,Zwiebeln gelb,Zwiebeln rot,funny-fr.Kicher.Papr,funnyfr. ungar.,iglo Rahmspinat
Transaktionsnummer,,,,,,,,,,,,,,,,,,,,,
399831,1,0,0,0,0,0,0.0,0,0,0,...,0,0,0.0,0,0,0,0,1,0,0
472857,0,0,0,0,0,0,0.0,0,0,0,...,0,0,0.0,1,0,0,0,0,0,0
486341,0,0,0,1,1,0,0.0,0,0,0,...,0,0,0.0,1,0,0,0,0,0,0
494783,0,0,0,0,0,0,0.0,0,0,0,...,0,0,0.0,0,1,0,0,0,0,0
508595,0,0,0,0,0,0,0.0,0,0,0,...,0,0,0.0,0,0,1,0,0,0,1
551374,0,0,0,0,0,0,0.0,0,0,0,...,0,0,0.0,0,0,0,0,0,0,0
576802,0,0,0,0,0,0,0.0,0,0,0,...,0,0,0.0,0,0,0,0,0,0,0
582831,0,1,0,0,0,0,0.0,0,0,0,...,0,0,0.0,0,0,1,0,0,0,0
597009,0,0,0,0,0,0,0.0,0,0,0,...,1,0,0.0,0,0,0,0,0,0,0


**Model Building**

In [57]:
# Building the model for Magdeburg dataset
frq_items_magdeburg = apriori(basket_Magdeburg, min_support = 0.06, use_colnames = True)

# Collecting the inferred rules in a dataframe
rules_magdeburg = association_rules(frq_items_magdeburg, metric ="lift", min_threshold = 1)
rules_magdeburg = rules_magdeburg.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules_magdeburg.head())

                                            antecedents  \
1132             (Pizza Mozzarella, Frische Weidemilch)   
1201                  (Sojajoghurt Natur, Buttergemüse)   
1203               (Gemüsemais ohne Salz, Buttergemüse)   
1341  (Sojajoghurt Natur, Buttergemüse, Romatomaten kg)   
1342  (Gemüsemais ohne Salz, Buttergemüse, Romatomat...   

                                 consequents  antecedent support  \
1132   (Paprika Chips, Active Sandwi. Toast)            0.064516   
1201  (Gemüsemais ohne Salz, Bio Haferdrink)            0.080645   
1203     (Sojajoghurt Natur, Bio Haferdrink)            0.080645   
1341  (Gemüsemais ohne Salz, Bio Haferdrink)            0.064516   
1342     (Sojajoghurt Natur, Bio Haferdrink)            0.064516   

      consequent support   support  confidence       lift  leverage  \
1132            0.096774  0.064516         1.0  10.333333  0.058273   
1201            0.096774  0.080645         1.0  10.333333  0.072841   
1203            0.09677

Applying model on Magdeburg dataset gives us some frequently purchased together products.

Apart from the fact that I mostly buy products that I want, to some extent the placement of some products did influence me at times to buy those when I went to pick up another product.

Some of these products according to the above association rules that were placed close (In each rule, bold ones are placed together):

(**Sojajoghurt Natur, Buttergemüse**) (Gemüsemais ohne Salz, **Bio Haferdrink**)

(Sojajoghurt Natur, Buttergemüse, **Romatomaten kg**) (**Gemüsemais ohne Salz**, Bio Haferdrink)

Further, all the association rules for this dataset align with my purchase behavior. It is interesting to see that I got habituated to purchase some products while I actually go to purchase other products.

In [ ]:
# Building the model for Aschaffenburg dataset
frq_items_aschaffenburg = apriori(basket_Aschaffenburg, min_support = 0.06, use_colnames = True)

# Collecting the inferred rules in a dataframe
rules_aschaffenburg = association_rules(frq_items_aschaffenburg, metric ="lift", min_threshold = 1)
rules_aschaffenburg = rules_aschaffenburg.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules_aschaffenburg.head())

                antecedents             consequents  antecedent support  \
114       (Zitrone Bio Stk)  (Bio Quinoa Tricolore)            0.111111   
115  (Bio Quinoa Tricolore)       (Zitrone Bio Stk)            0.111111   
120            (Gemüsemais)  (Bio.Dinkelmehl Vollk)            0.111111   
121  (Bio.Dinkelmehl Vollk)            (Gemüsemais)            0.111111   
128           (Marken Salz)  (Bio.Dinkelmehl Vollk)            0.111111   

     consequent support   support  confidence  lift  leverage  conviction  \
114            0.111111  0.111111         1.0   9.0  0.098765         inf   
115            0.111111  0.111111         1.0   9.0  0.098765         inf   
120            0.111111  0.111111         1.0   9.0  0.098765         inf   
121            0.111111  0.111111         1.0   9.0  0.098765         inf   
128            0.111111  0.111111         1.0   9.0  0.098765         inf   

     zhangs_metric  
114            1.0  
115            1.0  
120            1.0  
12

Applying model on Aschaffenburg dataset gives us some frequently purchased together products. As seen in EDA before, only few purchases were done in Aschaffenburg. So, finding proper association rules for this location through limited data is difficult.

Some of these products according to the above association rules that were placed close (In each rule, bold ones are placed close):
(**Zitrone Bio Stk**)  (**Bio Quinoa Tricolore**)